In [2]:
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional,BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.random import set_seed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tslearn.clustering import TimeSeriesKMeans
from sklearn.model_selection import train_test_split
import utils as utils
from importlib import reload
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)



2023-09-13 19:18:24.065217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 19:18:25.520939: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f6118be2cb0>
Traceback (most recent call last):
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, 

In [26]:

reload(utils)

# feature_list = ['PctChgClCl','sumPctChgClCl_2','sumPctChgClCl_3','sumPctChgClCl_4','sumPctChgClCl_5','sumPctChgClCl_6',
                # 'PctChgVol','sumPctChgVol_2','sumPctChgVol_3','sumPctChgVol_4','sumPctChgVol_5','sumPctChgVol_6',
                # 'PctChgVix','sumPctChgVix_2','sumPctChgVix_3','sumPctChgVix_4','sumPctChgVix_5','sumPctChgVix_6','rsi','Close','Open']
feature_list = ['PctChgClCl','sumPctChgClCl_2','sumPctChgClCl_3','sumPctChgClCl_4','sumPctChgClCl_5','sumPctChgClCl_6','Close']
n_steps = 75

features = len(feature_list)

tstart = 2005
tend = 2021

tickerList = ['aapl','spy','amd','qqq','amzn','meta','msft','f']
# tickerList = ['aapl','spy','amd']
# tickerList = ['aapl']

# Initialize arrays as empty arrays with the necessary number of dimensions
X_train, X_test = np.empty((0, n_steps, len(feature_list)-6)), np.empty((0, n_steps, len(feature_list)-6))
y_train, y_test = np.empty((0, 6)), np.empty((0, 6))

for ticker in tickerList: 
    df = utils.prepareStockDf(ticker,tstart)
    training_set, test_set = utils.train_test_split(df,tstart, tend, feature_list)

    dataset_total = df.loc[:,feature_list]
    test_set = dataset_total[len(dataset_total) - len(test_set) - n_steps :].values

    # scaler = MinMaxScaler(feature_range=(-1,1))
    # training_set = scaler.fit_transform(training_set)
    # test_set = scaler.transform(test_set)

    cur_X_train, cur_y_train = utils.split_sequence(training_set, n_steps)
    cur_X_test, cur_y_test = utils.split_sequence(test_set, n_steps)

    # Append cur_X_train, cur_X_test, cur_y_train, cur_y_test to the total arrays
    X_train = np.concatenate((X_train, cur_X_train), axis=0)
    X_test = np.concatenate((X_test, cur_X_test), axis=0)
    y_train = np.concatenate((y_train, cur_y_train), axis=0)
    y_test = np.concatenate((y_test, cur_y_test), axis=0)


display(X_train.shape)
display(y_train.shape)
display(X_test.shape)
display(y_test.shape)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

(31742, 75, 1)

(31742, 6)

(3368, 75, 1)

(3368, 6)

In [28]:
import math
n_clusters = math.ceil(math.sqrt(len(X_train))) // 3
# A good rule of thumb is choosing k as the square root of the number of points in the training data set in kNN

# km = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw")
km = TimeSeriesKMeans(n_clusters=n_clusters)

labels = km.fit_predict(X_train)

In [29]:
reload(utils)
threshold_factor = .5 # Adjust as necessary

X_train_adj, y_train_adj, labels_adj = utils.remove_outliers(X_train,y_train,labels,km,threshold_factor)
display(X_train.shape)
display(y_train.shape)
display(X_train_adj.shape)
display(y_train_adj.shape)
display(labels_adj.shape)

(31742, 75, 1)

(31742, 6)

(22912, 75, 1)

(22912, 6)

(22912,)

In [30]:
reload(utils)
utils.visualizeData(labels_adj,X_train_adj)


In [31]:
cluster_assignments = km.predict(X_test)
set(cluster_assignments)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58}

In [32]:
import pickle 
visualDict = {'labels': labels, 'X_train':X_train}
with open('visualDict2.pkl', 'wb') as file:
    pickle.dump(visualDict,file)

In [33]:
if len(y_train.shape) == 1:
    df = pd.DataFrame({'label': labels_adj, 'target': y_train_adj})
else:
    # Convert the 2D y_train into a DataFrame
    y_df = pd.DataFrame(y_train_adj, columns=[f'target_{i}' for i in range(y_train_adj.shape[1])])
    df = pd.concat([pd.DataFrame({'label': labels_adj}), y_df], axis=1)

df.head()
display(df['label'].value_counts())


8     1063
42     937
43     778
37     739
22     615
20     602
38     581
5      578
46     557
41     527
12     524
29     520
39     492
1      492
24     470
10     460
52     444
21     437
30     403
13     396
49     388
17     380
27     376
54     366
2      366
35     365
26     364
36     357
3      354
6      353
23     345
16     338
58     331
19     325
7      325
32     324
11     323
28     312
51     311
4      303
33     297
9      290
31     282
40     278
15     274
55     273
56     268
25     255
18     235
50     233
34     230
44     213
14     210
45     194
48     193
57     186
53     184
0      158
47     138
Name: label, dtype: int64

In [34]:
# display(X_train.shape)
# display(y_train.mean())
# display(cluster_X_train[0])

# label_filter = 1
# filtered_x_train, filtered_y_train = utils.filter_clusters(labels,label_filter,X_train_scaled,y_train_scaled)
# filtered_x_train.shape

In [35]:
def create_model():
    model_lstm = Sequential()

    # Using Bidirectional LSTM for the first layer
    model_lstm.add(Bidirectional(LSTM(units=100, activation='tanh', return_sequences=True), input_shape=(None, features-6)))
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(GRU(units=250, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(GRU(units=100, activation='tanh', return_sequences=True))  # Changed number of units to 250
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=100, activation='tanh'))  # Changed number of units to 100
    model_lstm.add(BatchNormalization())  # Batch normalization
    model_lstm.add(Dropout(0.2))

    model_lstm.add(Dense(units=6))

    # Using Adam optimizer with a learning rate scheduler
    optimizer = Adam(learning_rate=0.001)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=1e-5)

    model_lstm.compile(optimizer=optimizer, loss="mse")
    return model_lstm







In [36]:

for lab in set(labels_adj): 
    filtered_x_train, filtered_y_train = utils.filter_clusters(labels_adj, lab, X_train_adj, y_train_adj)

    cluster_assignments = km.predict(X_test)

    X_test_adj, y_test_adj, labels_test_adj = utils.remove_outliers(X_test,y_test,cluster_assignments,km,threshold_factor)

    filtered_x_test, filtered_y_test = utils.filter_clusters(labels_test_adj, lab, X_test_adj, y_test_adj)
    display(filtered_x_test.shape)
    display(filtered_y_test.shape)

    if len(filtered_x_test) == 0:
        continue

    display(filtered_x_train.shape)
    model_lstm = create_model()
    model_lstm.fit(filtered_x_train, filtered_y_train, batch_size=30, epochs=250, verbose=1)

    

    predicted_stock_price = model_lstm.predict(filtered_x_test)

    # Assuming predicted_stock_price has two columns
    results = pd.DataFrame({
        'predicted_1': predicted_stock_price[:, 0],
        'predicted_2': predicted_stock_price[:, 1],
        'predicted_3': predicted_stock_price[:, 2],
        'predicted_4': predicted_stock_price[:, 3],
        'predicted_5': predicted_stock_price[:, 4],
        'predicted_6': predicted_stock_price[:, 5],
        'real_1': filtered_y_test[:, 0],
        'real_2': filtered_y_test[:, 1],
        'real_3': filtered_y_test[:, 2],
        'real_4': filtered_y_test[:, 3],
        'real_5': filtered_y_test[:, 4],
        'real_6': filtered_y_test[:, 5]
    })

    # Assuming a correct prediction means both predicted features have the same sign as their corresponding real values
    results['same_1'] = ((results['predicted_1'] > 0) & (results['real_1'] > 0)) | ((results['predicted_1'] < 0) & (results['real_1'] < 0))
    results['same_2'] = ((results['predicted_2'] > 0) & (results['real_2'] > 0)) | ((results['predicted_2'] < 0) & (results['real_2'] < 0))
    results['same_3'] = ((results['predicted_3'] > 0) & (results['real_3'] > 0)) | ((results['predicted_3'] < 0) & (results['real_3'] < 0))
    results['same_4'] = ((results['predicted_4'] > 0) & (results['real_4'] > 0)) | ((results['predicted_4'] < 0) & (results['real_4'] < 0))
    results['same_5'] = ((results['predicted_5'] > 0) & (results['real_5'] > 0)) | ((results['predicted_5'] < 0) & (results['real_5'] < 0))
    results['same_6'] = ((results['predicted_6'] > 0) & (results['real_6'] > 0)) | ((results['predicted_6'] < 0) & (results['real_6'] < 0))
    accuracy1 = results['same_1'].sum() / len(results) * 100
    accuracy2 = results['same_2'].sum() / len(results) * 100
    accuracy3 = results['same_3'].sum() / len(results) * 100
    accuracy4 = results['same_4'].sum() / len(results) * 100
    accuracy5 = results['same_5'].sum() / len(results) * 100
    accuracy6 = results['same_6'].sum() / len(results) * 100

    # Construct the string
    output_string = (
        "Cluster Number: " + str(lab) +
        " \nAccuracy1D " + str(accuracy1) + " PredictedRet: " + str(results['predicted_1'].mean()) + " ActRet " + str(results['real_1'].mean() ) +
        " \nAccuracy2D " + str(accuracy2) + " PredictedRet: " + str(results['predicted_2'].mean()) + " ActRet " + str(results['real_2'].mean() ) +
        " \nAccuracy3D " + str(accuracy3) + " PredictedRet: " + str(results['predicted_3'].mean()) + " ActRet " + str(results['real_3'].mean() ) +
        " \nAccuracy4D " + str(accuracy4) + " PredictedRet: " + str(results['predicted_4'].mean()) + " ActRet " + str(results['real_4'].mean() ) +
        " \nAccuracy5D " + str(accuracy5) + " PredictedRet: " + str(results['predicted_5'].mean()) + " ActRet " + str(results['real_5'].mean() ) +
        " \nAccuracy6D " + str(accuracy6) + " PredictedRet: " + str(results['predicted_6'].mean()) + " ActRet " + str(results['real_6'].mean() ) +
        " Test set length: " + str(len(filtered_y_test)) + "\n"
    )

    # Write the string to a file
    with open('output2.txt', 'a') as f:
        f.write(output_string)






(10, 75, 1)

(10, 6)

(158, 75, 1)

2023-09-13 19:41:15.300096: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-13 19:41:15.301181: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-13 19:41:15.301883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/250


...

2023-09-13 22:07:06.179567: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-13 22:07:06.181157: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-13 22:07:06.182482: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/250


2023-09-13 22:07:07.443082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-13 22:07:07.444201: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-13 22:07:07.445060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

16/16 [==============================] - 14s 198ms/step - loss: 1.7884
Epoch 2/250
16/16 [==============================] - 3s 197ms/step - loss: 1.0601
Epoch 3/250
16/16 [==============================] - 3s 196ms/step - loss: 0.8961
Epoch 4/250
16/16 [==============================] - 3s 198ms/step - loss: 0.8558
Epoch 5/250
16/16 [==============================] - 4s 226ms/step - loss: 0.7190
Epoch 6/250
16/16 [==============================] - 5s 314ms/step - loss: 0.7022
Epoch 7/250
16/16 [==============================] - 5s 315ms/step - loss: 0.6881
Epoch 8/250
16/16 [==============================] - 5s 314ms/step - loss: 0.6961
Epoch 9/250
16/16 [==============================] - 5s 315ms/step - loss: 0.7099
Epoch 10/250
16/16 [==============================] - 5s 315ms/step - loss: 0.6807
Epoch 11/250
16/16 [==============================] - 5s 316ms/step - loss: 0.6150
Epoch 12/250
16/16 [==============================] - 5s 315ms/step - loss: 0.6070
Epoch 13/250
16/16 [====

KeyboardInterrupt: 

In [ ]:

lab = 17 
filtered_x_train, filtered_y_train = utils.filter_clusters(labels, lab, X_train_scaled, y_train_scaled)
filtered_x_train.shape
model_lstm = create_model()
model_lstm.fit(filtered_x_train, filtered_y_train, batch_size=30, epochs=200, verbose=0)

cluster_assignments = km.predict(X_test_scaled)
filtered_x_test, filtered_y_test = utils.filter_clusters(cluster_assignments, lab, X_test_scaled, y_test_scaled)

predicted_stock_price = model_lstm.predict(filtered_x_test)

# Assuming predicted_stock_price has two columns
results = pd.DataFrame({
    'predicted_1': predicted_stock_price[:, 0],
    'predicted_2': predicted_stock_price[:, 1],
    'predicted_3': predicted_stock_price[:, 2],
    'predicted_4': predicted_stock_price[:, 3],
    'predicted_5': predicted_stock_price[:, 4],
    'predicted_6': predicted_stock_price[:, 5],
    'real_1': filtered_y_test[:, 0],
    'real_2': filtered_y_test[:, 1],
    'real_3': filtered_y_test[:, 2],
    'real_4': filtered_y_test[:, 3],
    'real_5': filtered_y_test[:, 4],
    'real_6': filtered_y_test[:, 5]
})

# Assuming a correct prediction means both predicted features have the same sign as their corresponding real values
results['same_1'] = ((results['predicted_1'] > 0) & (results['real_1'] > 0)) | ((results['predicted_1'] < 0) & (results['real_1'] < 0))
results['same_2'] = ((results['predicted_2'] > 0) & (results['real_2'] > 0)) | ((results['predicted_2'] < 0) & (results['real_2'] < 0))
results['same_3'] = ((results['predicted_3'] > 0) & (results['real_3'] > 0)) | ((results['predicted_3'] < 0) & (results['real_3'] < 0))
results['same_4'] = ((results['predicted_4'] > 0) & (results['real_4'] > 0)) | ((results['predicted_4'] < 0) & (results['real_4'] < 0))
results['same_5'] = ((results['predicted_5'] > 0) & (results['real_5'] > 0)) | ((results['predicted_5'] < 0) & (results['real_5'] < 0))
results['same_6'] = ((results['predicted_6'] > 0) & (results['real_6'] > 0)) | ((results['predicted_6'] < 0) & (results['real_6'] < 0))
accuracy1 = results['same_1'].sum() / len(results) * 100
accuracy2 = results['same_2'].sum() / len(results) * 100
accuracy3 = results['same_3'].sum() / len(results) * 100
accuracy4 = results['same_4'].sum() / len(results) * 100
accuracy5 = results['same_5'].sum() / len(results) * 100
accuracy6 = results['same_6'].sum() / len(results) * 100

print("Cluster Number: " + str(lab) + 
    " \nAccuracy1D " + str(accuracy1) + " PredictedRet: " + str(results['predicted_1'].mean()) + " ActRet " + str(results['real_1'].mean() ) +
    " \nAccuracy2D " + str(accuracy2) + " PredictedRet: " + str(results['predicted_2'].mean()) + " ActRet " + str(results['real_2'].mean() ) +
    " \nAccuracy3D " + str(accuracy3) + " PredictedRet: " + str(results['predicted_3'].mean()) + " ActRet " + str(results['real_3'].mean() ) +
    " \nAccuracy4D " + str(accuracy4) + " PredictedRet: " + str(results['predicted_4'].mean()) + " ActRet " + str(results['real_4'].mean() ) +
    " \nAccuracy5D " + str(accuracy5) + " PredictedRet: " + str(results['predicted_5'].mean()) + " ActRet " + str(results['real_5'].mean() ) +
    " \nAccuracy6D " + str(accuracy6) + " PredictedRet: " + str(results['predicted_6'].mean()) + " ActRet " + str(results['real_6'].mean() ) + 
    " Test set length: " + str(len(filtered_y_test)))

NameError: name 'X_train_scaled' is not defined

In [ ]:

    # Get the cluster labels for the test data
cluster_assignments = km.predict(X_test_scaled)


NameError: name 'km' is not defined

In [12]:
unique_values = set(cluster_assignments)
unique_values

NameError: name 'cluster_assignments' is not defined

In [13]:
plot_count = math.ceil(math.sqrt(n_clusters))

fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
fig.suptitle('Clusters')
row_i=0
column_j=0
# For each label there is,
# plots every series with that label
for label in set(cluster_assignments):
    cluster = []
    for i in range(len(cluster_assignments)):
            if(cluster_assignments[i]==label):
                axs[row_i, column_j].plot(X_test[i],c="gray",alpha=0.4)
                cluster.append(X_test[i])
    if len(cluster) > 0:
        axs[row_i, column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
    axs[row_i, column_j].set_title("Cluster "+str(label))
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0
        
plt.show()

NameError: name 'n_clusters' is not defined

In [14]:
X_test.shape

NameError: name 'X_test' is not defined

In [15]:
cluster_X_test, cluster_y_test = utils.removeOutliers(X_test,y_test,cluster_assignments)
cluster_X_test.shape

NameError: name 'utils' is not defined

In [16]:
# filtered_x_test, filtered_y_test = utils.filter_clusters(cluster_assignments,label_filter,X_test_scaled,y_test_scaled)
filtered_x_test.shape

NameError: name 'filtered_x_test' is not defined

In [17]:


# padded_predictions = np.zeros((predicted_stock_price.shape[0], features))

# padded_predictions[:, 0] = predicted_stock_price.squeeze() 
# inverse_transformed_predictions = scaler.inverse_transform(padded_predictions)
# final_predictions = inverse_transformed_predictions[:, 0]

In [ ]:
predicted_stock_price = model_lstm.predict(filtered_x_test)
results = pd.DataFrame(filtered_y_test, predicted_stock_price.reshape(-1))
results = results.reset_index()
results.columns = ['predicted','real']
results['same'] = ((results['predicted'] > 0) & (results['real'] > 0)) | ((results['predicted'] < 0) & (results['real'] < 0))
results['same'].sum() / len(results) * 100
results

2/2 [==============================] - 0s 13ms/step


ValueError: Shape of passed values is (43, 6), indices imply (258, 6)

In [ ]:
results['same'] = ((results['predicted'] > 0) & (results['real'] > 0)) | ((results['predicted'] < 0) & (results['real'] < 0))
results['same'].sum() / len(results) * 100

48.03921568627451

In [ ]:
results.describe()

,predicted,real
count,102.000000,102.000000
mean,0.083790,0.061184
std,0.475419,0.559535
min,-1.017939,-1.000000
25%,-0.230329,-0.285544
50%,0.071836,0.088635
75%,0.398213,0.472033
max,1.053600,1.000000


In [ ]:
len(results[results['real'] > 0 ])

340